# Limpeza dos Dados do CAGR

## Importaçao dos Dados

In [1]:
import pandas as pd

df = pd.read_csv('Dados CAGR.csv',sep=';',index_col=0)

df = df.drop(['Ementa','Currículo'], axis=1)

## Correção de colunas com valores nulos por erro nos dados do CAGR:
map_cols = {
    'Horas  Aula':'HorasAula',
    'Vagas  Ofertadas':'VagasOfertadas', 
    'Vagas  Ocupadas':'VagasOcupadas', 
    'Alunos  Especiais':'AlunosEspeciais',
    'Saldo  Vagas':'SaldoVagas',
    'Pedidos  sem vaga':'Pedidossem vaga'
}

for col in map_cols.keys():
    df[col] = df[col].fillna(df[map_cols[col]])

df = df.drop(list(map_cols.values()), axis=1)

In [2]:
df.columns

Index(['Disciplina', 'Turma', 'Nome da Disciplina', 'Horas  Aula',
       'Vagas  Ofertadas', 'Vagas  Ocupadas', 'Alunos  Especiais',
       'Saldo  Vagas', 'Pedidos  sem vaga', 'Horários', 'Professor'],
      dtype='object')

In [ ]:
# As 20 matérias com mais pedidos sem vaga

df.groupby('Disciplina').sum().sort_values("Pedidos  sem vaga", ascending=False).head(20)

In [31]:
def get_centro(d):
    return d[0:3]

df["Centro"] = df["Disciplina"].apply(get_centro)

ListaDepto = df["Centro"].unique()
#df.groupby('Centro').sum().sort_values("Pedidos  sem vaga", ascending=False)
ListaDepto[0]

'ACL'

In [35]:
ListaDepto
Deptos=pd.DataFrame({'Centro':ListaDepto[:]})
#Deptos.to_csv("Departamentos.csv", sep=";")

## Extração dos Dados de Dia da Semana, Horário, Créditos e Local das Matérias

In [ ]:
def get_horarios(h):
    if isinstance(h, str):
        return h.replace(' / ','|').split(' ')
    else:
        return []

df["Limpeza Horário"] = df['Horários'].apply(get_horarios)

df["Limpeza Horário"]

def get_weekday(h):
    if len(h) >= 1:
        return [x[0] for x in h if len(x) > 0]
    else:
        return None

def get_time(h):
    if len(h) >= 1:
        return [x[2:6] for x in h if len(x) > 0]
    else:
        return None
    
def get_creditos(h):
    if len(h) >= 1:
        return [x[7] for x in h if len(x) > 7]
    else:
        return None
    
def get_local(h):
    if len(h) >= 1:
        return [x.split('|')[-1] for x in h if len(x) > 0]
    else:
        return None
    
    

df["Dias da Semana"] = df["Limpeza Horário"].apply(get_weekday)

df["Horários no dia"] = df["Limpeza Horário"].apply(get_time)

df["Créditos nos horários"] = df["Limpeza Horário"].apply(get_creditos)

df["Sala"] = df["Limpeza Horário"].apply(get_local)

df = df.drop(["Limpeza Horário"], axis='columns')

df.head(5)

## Mapeamento dos Horários para Análise BI

In [ ]:
horarios_df = pd.DataFrame()

lista_horarios = ["0730","0820","0910","1010","1100","1330","1420","1510","1620","1710","1830","1920","2020","2110"]
lista_dias_da_semana = ["2\.","3\.","4\.","5\.","6\.","7\."]
horarios_df["Disciplina"] = df["Disciplina"]
horarios_df["Turma"] = df["Turma"]


for dia in lista_dias_da_semana:
    horarios_df[dia] = df["Horários"].str.count(dia).fillna(0)

for hora in lista_horarios:
    horarios_df[hora] = df["Horários"].str.count(hora).fillna(0)
    

for dia in lista_dias_da_semana:
    for hora in lista_horarios:
        horarios_df[dia+hora] = df["Horários"].str.count(dia+hora).fillna(0)

new_cols = []
for col in horarios_df.columns:
    new_cols.append(col.replace('\.','.'))

horarios_df.columns = new_cols

horarios_df.head()

## Salvar arquivos CSV

In [ ]:
df.to_csv("Dados Tratados - CAGR.csv", sep=";")

horarios_df.to_csv("Dataframe dos Horários.csv", sep=";")

juntos = pd.merge(df,horarios_df)

juntos.to_csv("Join Dados Tratados + DF dos horários.csv", sep=";")

## Contagem de Turmas Oferecidas por Horário

In [ ]:
dias = [c.replace('\.','.') for c in lista_dias_da_semana]
grade = pd.DataFrame(index=lista_horarios, columns=dias)


for dia in dias:
    for hora in lista_horarios:
        grade.loc[hora][dia] = df["Horários"].str.count(dia+hora).fillna(0).sum().astype(int)
        
sns.heatmap(grade.fillna(0), annot=True, fmt="d")

## Contagem de Pedidos sem Vaga por Horário

In [ ]:
for dia in dias:
    for hora in lista_horarios:
        grade.loc[hora][dia] = df[pd.Series(df["Horários"].fillna("").str.contains(dia+hora))]["Pedidossem vaga"].sum().astype(int) 
                              
sns.heatmap(grade.fillna(0), annot=True, fmt="d")

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual

global view
view = df.copy()
global horarios
horarios = lista_horarios
global dias
def get_heatmaps(depto):
    
    grade_centros = pd.DataFrame(index=horarios, columns=dias)
    grade_pedidos = pd.DataFrame(index=horarios, columns=dias)
    
    view
    df = view.loc[view["Disciplina"].str.contains(depto)]
    
    # Subplot turmas oferecidas por horário
    for dia in dias:
        for hora in lista_horarios:
            grade_centros.loc[hora][dia] = df["Horários"].str.count(dia+hora).fillna(0).sum().astype(int)
    
    plt.figure(figsize=(18, 10))
    plt.subplot(1,2,1)
    sns.heatmap(grade_centros.fillna(0), annot=True, fmt="d", linewidth=0.2)
    plt.xticks(rotation=0)
    plt.yticks(rotation=0)
    plt.title("Mapa de Calor: Ofertas de Disciplinas")
    
    
    # Subplot pedidos sem vaga
    for dia in dias:
        for hora in lista_horarios:
            grade_pedidos.loc[hora][dia] = df[pd.Series(df["Horários"].fillna("").str.contains(dia+hora))]["Pedidos  sem vaga"].sum().astype(int) 
    plt.subplot(1,2,2)               
    sns.heatmap(grade_pedidos.fillna(0), annot=True, fmt="d", linewidth=0.2)
    plt.xticks(rotation=0)
    plt.yticks(rotation=0)
    plt.title("Mapa de Calor: Pedidos sem Vaga")
    

interact(get_heatmaps, depto='MTM')

In [ ]:
df.columns